# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [35]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [36]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# UnderSampling

In [37]:
from imblearn.under_sampling import RandomUnderSampler

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
y = y_data.values.reshape(-1, 1)

X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)


Counter({'ENFJ': 39,
         'ENFP': 39,
         'ENTJ': 39,
         'ENTP': 39,
         'ESFJ': 39,
         'ESFP': 39,
         'ESTJ': 39,
         'ESTP': 39,
         'INFJ': 39,
         'INFP': 39,
         'INTJ': 39,
         'INTP': 39,
         'ISFJ': 39,
         'ISFP': 39,
         'ISTJ': 39,
         'ISTP': 39})

# tokenize and stemming

In [38]:
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [39]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['want', 'know', 'feel', 'intp', 'actual', 'intp', 'idea', 'intp', 'havent', 'met', 'anyten', 'million', 'hug', 'mani', 'amaz', 'convers', 'held', 'absenc', 'vow', 'never', 'leav', 'realli', 'lot', 'written', 'recent', 'terriblyfacebook', 'way', 'emphas', 'other', 'error', 'cognit', 'becom', 'obviou', 'lead', 'wonder', 'im', 'friend', 'mani', 'peopl', 'rememb', 'peopl', 'moremmmmhm', 'ive', 'wonder', 'late', 'ei', 'think', 'definit', 'much', 'needwant', 'compani', 'around', 'regardless', 'time', 'togeth', 'spenthahah', 'love', 'think', 'that', 'go', 'topic', 'excel', 'convers', 'thank', 'particip', 'indulg', 'interest', 'enlighteningsomeon', 'creat', 'chitchat', 'thread', 'enfj', 'forum', 'enfj', 'vigilantli', 'await', 'type', 'wander', 'start', 'conversationswa', 'enfp', 'ive', 'caught', 'mani', 'text', 'look', 'around', 'shini', 'object', 'give', 'midsyl', 'mm', 'hmm', 'alway', 'mani', 'andi', 'tire', 'extravert', 'energysap', 'sentiment', 'arent', 'batshit', 

121

In [41]:
#vectorization (tfidf)
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData).toarray()
y = np.array(y_data)


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.05858548 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.04542902 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# XGBoost_Undersampling

In [42]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [43]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [44]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [45]:
import joblib

joblib.dump(model, 'XGBoost_undersample.pkl')

['XGBoost_undersample.pkl']

In [46]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.5
[0.66666667 0.54545455 0.45454545 0.16666667 0.8        0.29411765
 0.58333333 0.9        0.33333333 0.90909091 0.42857143 0.125
 0.42857143 0.42857143 0.47368421 0.53846154]
[0.75       0.46153846 0.625      0.2        0.5        0.45454545
 0.46666667 0.69230769 0.125      0.76923077 0.33333333 0.1
 0.6        0.6        0.5        0.63636364]
[0.70588235 0.5        0.52631579 0.18181818 0.61538462 0.35714286
 0.51851852 0.7826087  0.18181818 0.83333333 0.375      0.11111111
 0.5        0.5        0.48648649 0.58333333]
Evaluation END
